# Introduction
## Summary 

So far, we've talked about least squares regression using four models. 

$$
\begin{aligned}
\hat\mu &= \argmin_{m \in \mathcal{M}} \frac{1}{n} \sum_{i=1}^n \{ Y_i - m(X_i)\}^2 \quad \text{for} 
        && \mathcal{M} = \{ \text{increasing functions} \ m  \} \\
        &&& \mathcal{M} = \{ \text{convex functions} \ m \} \\
        &&& \mathcal{M} = \{ m \ : \ \rho_{\text{TV}}(m) \le B \} \\
        &&& \mathcal{M} = \{ m \ : \ \rho_{\text{Lip}}(m) \le B \}
\end{aligned}
$$

For every one, our implementation boiled down to two steps. Given a sample $\mathcal{X} = \{(X_1,Y_1),\dots,(X_n,Y_n)\}$, we do this. 
1. Think about what these constraints tell us about restriction $m_{\mid \mathcal{X}}$ of the functions in our model $\mathcal{M}$ to sample and choose the best of these. 
2. Extend this choice $\hat \mu_{\mid \mathcal{X}}$ to the real line in some way that keeps us in the model. 
  -  e.g. without increasing our seminorm $\rho$
  -  e.g.  violating our monotonicity or convexity constraints. 

This involves a bit of code but, with the help of CVXR, it's not too hard. Implementations are below. 
There's a lot of code there, but there's a lot of redundancy in it. If we wanted to boil this down to 50 or so lines, we could do that. All we'd have to do, really, is write more abstract fitting code and pass in the right constraints for each model. 

In [ ]:
suppressPackageStartupMessages({
  library(tidyverse)
  library(CVXR)
  CVXR::add_to_solver_blacklist('OSQP') 
})

## A function we'll want. unique with an inverse mapping.

invert.unique = function(x) { 
  o = order(x)
  dup = duplicated(x[o])
  inverse = rep(NA, length(x))
  inverse[o] = cumsum(!dup)
  list(elements=o[!dup], inverse=inverse)
}

a = c(1,2,3,3,4,5,5,5,6)
unique.a=invert.unique(a)
stopifnot(a[unique.a$elements][unique.a$inverse] == a)

## Fitting Functions

monotonereg = function(X, Y, decreasing = FALSE) {
  # Step 0.
  # We check that the inputs satisfy our assumptions.
  stopifnot(length(X) == length(Y))
  input = list(X=X, Y=Y)
  n = length(X)
  # and find the unique elements of X and the inverse mapping
  unique.X = invert.unique(X)

  # Step 1.
  # We tell CVXR we're thinking about a vector of unknowns m in R^p.
  m = Variable(length(unique.X$elements))
  # and permute and duplicate these into a vector mX with n elements in correspondence with (X_1,Y_1)...(X_n,Y_n)
  mX = m[unique.X$inverse]

  # Step 2.
  # We tell CVXR that we're interested in mean squared error.
  mse = sum((Y - mX)^2 / n)

  # Step 3.
  # We specify our constraints.
  constraints = list(if (decreasing) { diff(m) <= 0 } else { diff(m) >= 0 })

  # Step 4.
  # We ask CVXR to minimize mean squared error subject to our constraints.
  # And we ask for vector mu.hat that does it.
  solved = solve(Problem(Minimize(mse), constraints))
  mu.hat = solved$getValue(m)

  # Step 5: a little boilerplate to make it idiomatic R.
  # 1. we record the unique levels of X and mu.hat, in correspondence and sorted in increasing order of X, in a list. We also record the input data. 
  # 2. we assign that list a class, so R knows predict should delegate to predict.monotonereg
  # 3. we return the list
  model = list(X = X[unique.X$elements], mu.hat = mu.hat, input = input)
  attr(model, "class") = "monotonereg"
  model
}

convexreg = function(X, Y, concave = FALSE, monotone = NULL) {
  # Step 0.
  # We check that the inputs satisfy our assumptions.
  stopifnot(length(X) == length(Y))
  input = list(X=X, Y=Y)
  n = length(X)
  # and find the unique elements of X and the inverse mapping
  unique.X = invert.unique(X)

  # Step 1.
  # We tell CVXR we're thinking about a vector of unknowns m in R^p.
  m = Variable(length(unique.X$elements))
  # and permute and duplicate these into a vector mX with n elements in correspondence with (X_1,Y_1)...(X_n,Y_n)
  mX = m[unique.X$inverse]

  # Step 2.
  # We tell CVXR that we're interested in mean squared error.
  mse = sum((Y - mX)^2 / n)

  # Step 3.
  # We specify our constraints.
  # Interpretation (rearrange): secant slopes are increasing
  uX = X[unique.X$elements]
  ii = 1:(n-2)
  constraints = 
    list(((m[ii+1]-m[ii])  * (uX[ii+2]-uX[ii]) - 
          (m[ii+2]-m[ii]) *  (uX[ii+1]-uX[ii])) * (-1)^concave <= 0)
  if(!is.null(monotone)) { 
       decreasing = monotone == 'decreasing'
       constraints = c(constraints, diff(m) * (-1)^decreasing >= 0)
  }
  # Step 4.
  # We ask CVXR to minimize mean squared error subject to our constraints.
  # And we ask for vector mu.hat that does it.
  solved = solve(Problem(Minimize(mse), constraints))
  mu.hat = solved$getValue(m)

  # Step 5: a little boilerplate to make it idiomatic R.
  # 1. we record the unique levels of X and mu.hat, in correspondence and sorted in increasing order of X, in a list. We also record the input data. 
  # 2. we assign that list a class, so R knows predict should delegate to predict.convexreg
  # 3. we return the list
  model = list(X = X[unique.X$elements], mu.hat = mu.hat, input = input)
  attr(model, "class") = "convexreg"
  model
}

bvreg = function(X, Y, B=1) {
  # Step 0.
  # We check that the inputs satisfy our assumptions.
  stopifnot(length(X) == length(Y))
  input = list(X=X, Y=Y)
  n = length(X)
  # and find the unique elements of X and the inverse mapping
  unique.X = invert.unique(X)

  # Step 1.
  # We tell CVXR we're thinking about a vector of unknowns m in R^p.
  m = Variable(length(unique.X$elements))
  # and permute and duplicate these into a vector mX with n elements in correspondence with (X_1,Y_1)...(X_n,Y_n)
  mX = m[unique.X$inverse]

  # Step 2.
  # We tell CVXR that we're interested in mean squared error.
  mse = sum((Y - mX)^2 / n)

  # Step 3.
  # We specify our constraints.
  constraints = list( sum(abs(diff(m))) <= B )

  # Step 4.
  # We ask CVXR to minimize mean squared error subject to our constraints.
  # And we ask for vector mu.hat that does it.
  solved = solve(Problem(Minimize(mse), constraints))
  mu.hat = solved$getValue(m)

  # Step 5: a little boilerplate to make it idiomatic R.
  # 1. we record the unique levels of X and mu.hat, in correspondence and sorted in increasing order of X, in a list. We also record the input data. 
  # 2. we assign that list a class, so R knows predict should delegate to predict.bvreg
  # 3. we return the list
  model = list(X = X[unique.X$elements], mu.hat = mu.hat, B=B, input = input)
  attr(model, "class") = "bvreg"
  model
}

lipreg = function(X, Y, B=1) {
  # Step 0.
  # We check that the inputs satisfy our assumptions.
  stopifnot(length(X) == length(Y))
  input = list(X=X, Y=Y)
  n = length(X)
  # and find the unique elements of X and the inverse mapping
  unique.X = invert.unique(X)

  # Step 1.
  # We tell CVXR we're thinking about a vector of unknowns m in R^p.
  m = Variable(length(unique.X$elements))
  # and permute and duplicate these into a vector mX with n elements in correspondence with (X_1,Y_1)...(X_n,Y_n)
  mX = m[unique.X$inverse]

  # Step 2.
  # We tell CVXR that we're interested in mean squared error.
  mse = sum((Y - mX)^2 / n)

  # Step 3.
  # We specify our constraints.
  uX = X[unique.X$elements]
  constraints = list( abs(diff(m)) <= B * diff(uX) )

  # Step 4.
  # We ask CVXR to minimize mean squared error subject to our constraints.
  # And we ask for vector mu.hat that does it.
  solved = solve(Problem(Minimize(mse), constraints))
  mu.hat = solved$getValue(m)

  # Step 5: a little boilerplate to make it idiomatic R.
  # 1. we record the unique levels of X and mu.hat, in correspondence and sorted in increasing order of X, in a list. We also record the input data. 
  # 2. we assign that list a class, so R knows predict should delegate to predict.lipreg
  # 3. we return the list
  model = list(X = X[unique.X$elements], mu.hat = mu.hat, B=B, input = input)
  attr(model, "class") = "lipreg"
  model
}

## Prediction Functions

# make predictions based on piecewise-constant interpolation
# we use the curve that jumps at each observation and is otherwise constant
# that is, if X[1] < X[2] < ..., 
#   mu.hat(x) for x between X[k] and X[k+1] is mu.hat(X[k])   [case 1]
#             for x > X[k]  is mu.hat(X[k])                   [case 2]
#             for x < X[1]  is mu.hat(X[1])                   [case 3]
predict.piecewise.constant = function(model, newdata=data.frame(X=model$input$X)) {
  Y = model$mu.hat; X=model$X; x=newdata$X
  # for each new data point x in newdata$X, 
  # find the closest observed X[k] left of x
  # i.e., the largest k for which X[k] <= x 
  # this covers cases 1 and 2
  # i will be a vector of these numbers k, with one for each x in newdata$X
  i = findInterval(x, X) 
  # if there is no X[k] < x, findInterval tells us k=0
  # to cover case 3, we want X[k] for k=1 when this happens.
  i[i==0] = 1
  # report the values of mu.hat(X[k]), one for each x
  Y[i]
}

predict.piecewise.linear = function(model, newdata=data.frame(X=model$input$X)) {
  Y = model$mu.hat; X=model$X; x=newdata$X; n = length(X) 
  # for each new data point x[k]
  # find the closest observed X[i[k]] left of x[k]
  # i.e., i[k] is the largest integer i for which X[i] <= x[k] 
  i = findInterval(newdata$X, X) 
  # If there is no X[i] < x[k], findInterval sets i[k]=0
  #  and we'll want to act as if we'd gotten 1 so we use the
  #  line through (X[1], Y[1])  and (X[2], Y[2])
  # If that k is n, we'll want to act as if we'd gotten n-1 so we use 
  #  the line through (X[n-1], Y[n-1])  and (X[n], Y[n])
  i[i==0] = 1; i[i==n] = n-1
  # make a prediction using the formula y - y0 = (x-x0) * slope 
  Y[i] + (x-X[i]) * (Y[i+1]-Y[i])/(X[i+1]-X[i])
}

predict.monotonereg = predict.piecewise.constant
predict.convexreg = predict.piecewise.linear
predict.bvreg = predict.piecewise.constant
predict.lipreg = predict.piecewise.linear

## Conveniences

prediction.function = function(model) { 
  if(class(model)=='function') { model }
  else { function(x) { predict(model, newdata=data.frame(X=x)) } }
}

## Styles

lab.theme = theme(plot.background = element_rect(fill = "transparent", colour = NA),
		    					panel.background = element_rect(fill = "transparent", colour = NA),
                  legend.background = element_rect(fill="transparent", colour = NA),
                  legend.box.background = element_rect(fill="transparent", colour = NA),
                  legend.key = element_rect(fill="transparent", colour = NA),
				panel.grid.major=element_line(color=rgb(0,0,0,.05,   maxColorValue=1)),
	      panel.grid.minor=element_line(color=rgb(0,0,0,.02,   maxColorValue=1)),
		    axis.ticks.x = element_blank(),
		    axis.ticks.y = element_blank(),
		    axis.text.x  = element_text(colour = "#aaaaaa"),
		    axis.text.y  = element_text(colour = "#aaaaaa"),
		    axis.title.x  = element_text(colour = "#aaaaaa"),
		    axis.title.y  = element_text(colour = "#aaaaaa", angle=90))
theme_set(lab.theme)


Here's an example where they all work. If you uncomment line 4, most of them don't. 

In [ ]:
set.seed(1)
n = 50
mu = function(x) { x } 
#mu = function(x) { -x^2 }
X = runif(n)
Y = mu(X) + .25*rnorm(n)

models = list(monotone = monotonereg(X, Y),
              convex = convexreg(X, Y),
              bv = bvreg(X, Y),
              lip = lipreg(X, Y)) |> map(prediction.function)

x = seq(0, 1, by=.01) 
ggplot() + 
  geom_point(data=data.frame(X, Y), aes(X, Y), color='black', alpha=.3) + 
  geom_line(data=data.frame(x=x, y=mu(x)), aes(x, y), color='black', linewidth=2, alpha=.3) + 
  geom_line(data=data.frame(x=x, y=models$monotone(x)), aes(x, y), color='red', alpha=.3) + 
  geom_line(data=data.frame(x=x, y=models$convex(x)), aes(x, y), color='blue', alpha=.3) + 
  geom_line(data=data.frame(x=x, y=models$bv(x)), aes(x, y), color='green', alpha=.3) + 
  geom_line(data=data.frame(x=x, y=models$lip(x)), aes(x, y), color='purple', alpha=.3)


## What are these models? 

In differentiable terms ...
$$
\begin{aligned}
\mathcal{M} &= \{ m \ : \ m'(x) \ge 0 \}   && \text{monotone} \\
\mathcal{M} &= \{ m \ : \ m''(x) \ge 0 \}  && \text{convex}  \\
\mathcal{M} &= \{ m \ : \lVert m'(x) \rVert_{L_1} \le B \}  && \text{bounded variation} \\
\mathcal{M} &= \{ m \ : \lVert m'(x) \rVert_{L_\infty} \le B \}  && \text{Lipschitz} \\
\end{aligned}
$$
But if we tried to optimize over sets of differentiable functions like this, we'd converge to something nondifferentiable. 
So we use definitions that don't require differentiability instead.  

- The **monotone** case is easy enough to generalize. Probably easier without the derivative.
$$
\begin{aligned}
\mathcal{M} &= \{ m \ :  \ m(a) \le m(b) \ \forall a \le b \}  
\end{aligned}
$$

- The **bounded variation** case we covered in class. It took a little thought, but the result was simple. 
$$
\begin{aligned}
\mathcal{M} &= \{ m \ :  \rho_{TB}(m) \le B  \} \quad 
\text{for} \quad \rho_{TV}(m)=\sup_{\substack{\text{increasing sequences } \\
0=x_1 < x_2 < \dots < x_n=1 }} \sum_{i=1}^{n-1} |m(x_{i+1}) - m(x_i)|
\end{aligned}
$$

- The other two are about the slopes of *secants*. Let's think this through.
- The differentiable curve below, $m(x)=x^3/3$, is *convex* with $\lVert m'(x) \rVert_{L_\infty} = 1$.  
  - What do our differentiable-case definitions tell us about the slopes of *tangents* to this curve? 
  - What does this tell us about the slopes of *secants*?

In [ ]:
mu = function(x) { x^3/3 } 
ggplot() + geom_line(aes(x=x, y=mu(x)), linewidth=2, alpha=.5) 

*Tip*. First, think about the secant between $x$ and $x+h$ for small $h$. Then, think about the mean value theorem. 

### The usual definition of convexity

The usual definition of convexity isn't about secant slopes. It's about secants lying above the curve. $m$ is convex if and only if, for all $a \le b$,

$$
(1-\lambda)m(a) + \lambda m(b) \ge m\{(1-\lambda)a + \lambda b\} \quad \text{for all } \lambda \in [0,1]
$$

What does that mean in plot terms? Draw it! 

How is this related to secant slopes? Well, let's transform it into a statement about slopes. 
Let's think about doing that to the right-hand side of the inequality above. But obviously,
to get an equivalent inequality, we need to apply the same transformation to the left-hand side. 

To get a secant slope, we need to ...
1. Get a 'rise' by subtracting $m(x)$ at some $x$ . I like $m(a)$. 
2. Divide the corresponding 'run' 

What does the resulting inequality mean? Draw it!

### Next Class

We'll talk about Sobolev models.
$$
\begin{aligned}
\mathcal{M} &= \{ m \ : \ \lVert m' \rVert_{L_2} \le B \}  && \text{Sobolev} \\
\end{aligned}
$$
This is a lot like the Lipchitz and BV models, but we'll use pretty different tools to analyze them.
- We'll think about $\frac{d^2}{dx^2}$ as a self-adjoint operator on periodic functions. 
- And use the eigenvalues and eigenfunctions of this operator to understand it.
- Should we review that stuff from the inner product spaces homework today?

## Examples

Let's look at some examples from the homework.

# Implementation

## Restriction to the Sample

## Fitting the Slow Way 

## Extension to the Real Line

### Something that doesn't work. 
Why doesn't piecewise-constant extension work for the Lipschitz and convex models? 

### Something that does. 
**Common key idea**. If we have a piecewise-linear curve, each secant slope is a weighted average of segment slopes.

1. Why does this imply the piecewise-linear extension of a $B$-Lipschitz function on $\mathcal{X}$ is $B$-Lipschitz?
2. Why does this imply the piecewise-linear extension of a convex function is convex? 

How does this tell us extending our restricted solution to the real line gives us a solution to the unrestricted problem?

# Optimized Implementation

In a naive implementation, we get *a lot* of constraints. Roughly how many?
  - Lipschitz?
  - Convex?
  
How can we reduce the number of constraints? Think about what *local* properties tell us about the *global* behavior.  

- Lipschitz case. 
- Convex case. Why is it a little less simple? 


# Rates of Convergence 

In [ ]:
ns = c(25, 50,100,200,400,800,1600)
x = seq(0,1,by=.001)

mu = function(x) { x }
sigma = .5

models  = list(lines = \(X,Y)lm(Y~X),
               monotone = monotonereg,
	             bv         = bvreg,
               lip        = lipreg,
               convex     = convexreg,
               convex.monotone = \(X,Y) convexreg(X,Y,monotone='increasing'))
               
errors = list(sample     = function(mu.hat, mu, X) { mean( (mu.hat(X)-mu(X))^2 ) },
	            population = function(mu.hat, mu, X) { mean( (mu.hat(x)-mu(x))^2 ) })

tabulate.errors.for.sample = function(X,Y,mu,ns) { 
  purrr::map_dfr(ns, function(n) {
    purrr::map_dfr(models, .id='model', function(model.fit) {
      tryCatch({
        model = model.fit(X[1:n], Y[1:n])
        purrr::map_dfr(errors, .id='error.measure', function(error) {
          data.frame(n=n, error=error(prediction.function(model), mu, X[1:n]))
        })
      }, error = function(e) {})
    })
  })
}

tabulate.errors = function(models, replications=20) { 
  purrr::map_dfr(1:replications, .id='rep', function(rep) {
    X           = runif(max(ns))
    epsilon     = sigma*rnorm(length(X))
    Y = mu(X) + epsilon
    tabulate.errors.for.sample(X, Y, mu, ns)
  })
}

set.seed(2)
tab = tabulate.errors(models, replications=10)

In [ ]:
rates = tab %>% 
    group_by(error.measure, model) %>% 
	group_modify(function(group,...) {
	    reg.data = group %>% group_by(n) %>% summarize(error=mean(error))
	    model = nls(formula = sqrt(error) ~ a*n^(-b), data=reg.data, 
	                start=list(a=1, b=1/2))
	     data.frame(a=coef(model)[1], b=coef(model)[2])
	})

rate.predicted = tab %>% 
    group_by(model,error.measure) %>% 
	group_modify(function(group,...) {
	    reg.data = group %>% group_by(n) %>% summarize(error=mean(error))
	    model = nls(formula = sqrt(error) ~ a*n^(-b), data=reg.data, 
	    	        start=list(a=1, b=1/2))
	    reg.data$error = predict(model)^2
	    reg.data
	})


plot.error.curves = function(error.measure) { 
    ggplot(tab[tab$error.measure==error.measure,], 
       aes(x=n, y=error, color=model)) + 
    stat_summary(geom='line', fun=mean) +  
    stat_summary(geom='pointrange', fun.data=mean_se,
    		 position=position_dodge(5)) 
}
plot.error.curves.with.rate = function(error.measure) { 
    plot.error.curves(error.measure) +
    geom_line(aes(x=n, y=error, color=model), alpha=.4, linewidth=2, 
	data=rate.predicted[rate.predicted$error.measure == error.measure, ])
}

plot.error.curves.with.rate('population') + coord_cartesian(ylim=c(0,.01))
rates

- Note differences in rate between 
  - Lines 
  - Convex 
  - Monotone, Bounded Variation, and Lipschitz

- What's that about? Intuition? 
  - Note: monotone convex and convex are get pretty similar rates. 
  - So you can think of the convex model as 'smaller' than the monotone model.  
  - But the Lipschitz model is 'smaller' than the 'Bounded Variation' model. Why no difference in rate there?

- Maybe a constraint on seconnd derivative stonger than a constraint on the first 
   - e.g. Think about what taylor approximation tells you when you have 1 vs. 2 bounded derivatives.
   - e.g. Think about a way of counting 'meaningfully different' increasing curves. What proportion of them are convex?